In [14]:
from geopy.geocoders import Nominatim, ArcGIS, MapBox
import pandas as pd
import geopandas as gpd
import googlemaps
from functools import partial
#from tqdm import tqdm
from time import sleep
from dotenv import load_dotenv
from dotenv import dotenv_values
import os

In [18]:
path = r"C:\Users\luism\Documents\Luis Code\CSV\merged_camps1.csv"
path2 = r"C:\Users\luism\Documents\Luis Code\CSV\merged_camps1.csv"
csv1 = pd.read_csv(path)
csv2 = pd.read_csv(path2)
csv_copy = csv1.copy()
csv1_copy = csv1.copy()

load_dotenv()
mapbox_key = os.getenv("MAPBOX_API_KEY")
# i had to get the Google API manually from the env because the code don't recognize with method above. the same i use with Mapbox
config = dotenv_values(".env")
google_maps_api_key = config.get("GOOGLE_MAPS_API_KEY")




Thanks for the code! - [geografope](https://github.com/geografope/geocodificacion-con-python/blob/main/geocoding.ipynb)

In [24]:
def concatenar_campos(row):
    return f"{row['address']}, {row['city']}, {row['state']}, {row['zip_code']}, USA"
     

csv_copy['nogeo'] = csv_copy.apply(concatenar_campos, axis= 1)
csv1_copy['nogeo'] = csv1_copy.apply(concatenar_campos, axis= 1)
csv_copy.head()

,camp_name,address,category,director,operator,city,state,accredited-member,programs,zip_code,nogeo
0,URJ Eisner Camp,53 Brookside Rd,"Jewish, Reform",NaN,URJ Eisner and Crane Lake Camp (Operator),Great Barrington,MA,Accredited Member,4.0,01230-2186,"53 Brookside Rd, Great Barrington, MA, 01230-2..."
1,Camp Chateaugay,233 Gadway Rd,"Independent, For Profit",Mitch Goldman,"Chateaugay Campco, LLC (Operator)",Merrill,NY,Accredited Member,2.0,12955-2307,"233 Gadway Rd, Merrill, NY, 12955-2307, USA"
2,YWCA VT Camp Hochelaga,34 Hochelaga Rd,YWCA,Christine Perry,YWCA Vermont,South Hero,VT,Accredited Member,2.0,05486-4807,"34 Hochelaga Rd, South Hero, VT, 05486-4807, USA"
3,YMCA Camp Ohiyesa,7300 Hickory Ridge Rd,YMCA,Doug Grimm,YMCA Camping Services of Detroit,Holly,MI,Accredited Member,4.0,48442-8929,"7300 Hickory Ridge Rd, Holly, MI, 48442-8929, USA"
4,Camp CUMCITO,13220 Mission Rd,Nondenominational Christian,NaN,City Union Mission,Warsaw,MO,Accredited Member,4.0,13220,"13220 Mission Rd, Warsaw, MO, 13220, USA"


## **Using Mapbox API**

In [25]:
api_mapbox = mapbox_key
geolocator = MapBox(api_key = api_mapbox)
def tidygeocode(row, index, total_rows):
    location = geolocator.geocode(row['nogeo'])
    if location:
        lat, lon = location.latitude, location.longitude
        print(f"Processed {index + 1}/{total_rows}: {row['nogeo']} -> {lat}, {lon}")
        return pd.Series({'latitude': lat, 'longitude': lon})
    else:
        print(f"Processed {index + 1}/{total_rows}: {row['nogeo']} -> No coordinates found")
        return pd.Series({'latitude': None, 'longitude': None})

# Aplicar la función fila por fila con progreso
total_rows = len(csv_copy)  # Total de filas
results = []

for index, row in csv_copy.iterrows():
    result = tidygeocode(row, index, total_rows)
    results.append(result)



csv_copy.head()
     

Processed 1/2270: 53 Brookside Rd, Great Barrington, MA, 01230-2186, USA -> 42.177243, -73.353531
Processed 2/2270: 233 Gadway Rd, Merrill, NY, 12955-2307, USA -> 44.76557, -73.982062
Processed 3/2270: 34 Hochelaga Rd, South Hero, VT, 05486-4807, USA -> 44.607093, -73.297982
Processed 4/2270: 7300 Hickory Ridge Rd, Holly, MI, 48442-8929, USA -> 42.702335, -83.661508
Processed 5/2270: 13220 Mission Rd, Warsaw, MO, 13220, USA -> 38.235615, -93.463336
Processed 6/2270: 484 Camp Orkila Rd, Eastsound, WA, 98245-5071, USA -> 48.70037, -122.94095
Processed 7/2270: 4180 E Emigration Canyon Rd, Salt Lake City, UT, 84108-1764, USA -> 40.76451, -111.78038
Processed 8/2270: PO Box 770100, Coral Springs, FL, 33077-0100, USA -> 29.583264, -81.210136
Processed 9/2270: 7303 Walker Rd, Carsonville, MI, 48419-9492, USA -> 43.387874, -82.5372
Processed 10/2270: 1724 W Main St, Lake Geneva, WI, 53147-1652, USA -> 42.586695, -88.451069
Processed 11/2270: 12100 Seminole Pratt Whitney Rd, West Palm Beach, FL

,camp_name,address,category,director,operator,city,state,accredited-member,programs,zip_code,nogeo
0,URJ Eisner Camp,53 Brookside Rd,"Jewish, Reform",NaN,URJ Eisner and Crane Lake Camp (Operator),Great Barrington,MA,Accredited Member,4.0,01230-2186,"53 Brookside Rd, Great Barrington, MA, 01230-2..."
1,Camp Chateaugay,233 Gadway Rd,"Independent, For Profit",Mitch Goldman,"Chateaugay Campco, LLC (Operator)",Merrill,NY,Accredited Member,2.0,12955-2307,"233 Gadway Rd, Merrill, NY, 12955-2307, USA"
2,YWCA VT Camp Hochelaga,34 Hochelaga Rd,YWCA,Christine Perry,YWCA Vermont,South Hero,VT,Accredited Member,2.0,05486-4807,"34 Hochelaga Rd, South Hero, VT, 05486-4807, USA"
3,YMCA Camp Ohiyesa,7300 Hickory Ridge Rd,YMCA,Doug Grimm,YMCA Camping Services of Detroit,Holly,MI,Accredited Member,4.0,48442-8929,"7300 Hickory Ridge Rd, Holly, MI, 48442-8929, USA"
4,Camp CUMCITO,13220 Mission Rd,Nondenominational Christian,NaN,City Union Mission,Warsaw,MO,Accredited Member,4.0,13220,"13220 Mission Rd, Warsaw, MO, 13220, USA"


In [27]:
api_key = google_maps_api_key
gmaps = googlemaps.Client(key=api_key)



# Función para geocodificar direcciones
def geocode_address(address):
    try:
        geocode_result = gmaps.geocode(address)
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

# Aplicar la función a cada fila del DataFrame
latitudes = []
longitudes = []

for index, row in csv1_copy.iterrows():
    lat, lng = geocode_address(row['nogeo'])
    latitudes.append(lat)
    longitudes.append(lng)
    print(f"Processed {index + 1}/{len(csv1_copy)}: {row['nogeo']} -> {lat}, {lng}")
    sleep(0.1)  # Para evitar exceder el límite de solicitudes por segundo

# Agregar columnas de latitud y longitud al DataFrame
csv1_copy['latitude'] = latitudes
csv1_copy['longitude'] = longitudes

print("Geocoding completed!")

Processed 1/2270: 53 Brookside Rd, Great Barrington, MA, 01230-2186, USA -> 42.1759904, -73.3568677
Processed 2/2270: 233 Gadway Rd, Merrill, NY, 12955-2307, USA -> 44.766374, -73.98201879999999
Processed 3/2270: 34 Hochelaga Rd, South Hero, VT, 05486-4807, USA -> 44.60583, -73.29696609999999
Processed 4/2270: 7300 Hickory Ridge Rd, Holly, MI, 48442-8929, USA -> 42.704443, -83.662075
Processed 5/2270: 13220 Mission Rd, Warsaw, MO, 13220, USA -> 38.236238, -93.46269099999999
Processed 6/2270: 484 Camp Orkila Rd, Eastsound, WA, 98245-5071, USA -> 48.7003671, -122.9409952
Processed 7/2270: 4180 E Emigration Canyon Rd, Salt Lake City, UT, 84108-1764, USA -> 40.764514, -111.7803837
Processed 8/2270: PO Box 770100, Coral Springs, FL, 33077-0100, USA -> 26.2369447, -80.2437255
Processed 9/2270: 7303 Walker Rd, Carsonville, MI, 48419-9492, USA -> 43.38793709999999, -82.537255
Processed 10/2270: 1724 W Main St, Lake Geneva, WI, 53147-1652, USA -> 42.5864813, -88.4518957
Processed 11/2270: 12100

In [28]:
csv1_copy.to_csv("camps_11.csv", index=False)

In [29]:
csv1_copy.head()

,camp_name,address,category,director,operator,city,state,accredited-member,programs,zip_code,nogeo,latitude,longitude
0,URJ Eisner Camp,53 Brookside Rd,"Jewish, Reform",NaN,URJ Eisner and Crane Lake Camp (Operator),Great Barrington,MA,Accredited Member,4.0,01230-2186,"53 Brookside Rd, Great Barrington, MA, 01230-2...",42.175990,-73.356868
1,Camp Chateaugay,233 Gadway Rd,"Independent, For Profit",Mitch Goldman,"Chateaugay Campco, LLC (Operator)",Merrill,NY,Accredited Member,2.0,12955-2307,"233 Gadway Rd, Merrill, NY, 12955-2307, USA",44.766374,-73.982019
2,YWCA VT Camp Hochelaga,34 Hochelaga Rd,YWCA,Christine Perry,YWCA Vermont,South Hero,VT,Accredited Member,2.0,05486-4807,"34 Hochelaga Rd, South Hero, VT, 05486-4807, USA",44.605830,-73.296966
3,YMCA Camp Ohiyesa,7300 Hickory Ridge Rd,YMCA,Doug Grimm,YMCA Camping Services of Detroit,Holly,MI,Accredited Member,4.0,48442-8929,"7300 Hickory Ridge Rd, Holly, MI, 48442-8929, USA",42.704443,-83.662075
4,Camp CUMCITO,13220 Mission Rd,Nondenominational Christian,NaN,City Union Mission,Warsaw,MO,Accredited Member,4.0,13220,"13220 Mission Rd, Warsaw, MO, 13220, USA",38.236238,-93.462691
